# AutoGen AgentChat Quickstart

This notebook provides a quick introduction to AutoGen's AgentChat, adapted for SurfSense integration.

Based on the official [AutoGen AgentChat Quickstart](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html)

## What is AgentChat?

AgentChat is a high-level API in AutoGen for building multi-agent applications. It provides:
- Pre-built agent types for common scenarios
- Built-in group chat coordination
- Easy integration with LLMs
- Code execution capabilities
- Human-in-the-loop support

## Prerequisites

```bash
pip install pyautogen
```

In [ ]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

## Configuration

First, set up your LLM configuration. AutoGen supports multiple LLM providers.

In [ ]:
# Set your API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Configure the LLM
config_list = [
    {
        "model": "gpt-4",
        "api_key": os.environ.get("OPENAI_API_KEY"),
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0,
}

## Example 1: Basic Two-Agent Conversation

The simplest AutoGen setup: an assistant agent and a user proxy agent.

In [ ]:
# Create an assistant agent
assistant = AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    system_message="You are a helpful AI assistant."
)

# Create a user proxy agent
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  # No human input needed for this example
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

# Start the conversation
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about programming."
)

## Example 2: Code Execution

AutoGen agents can write and execute code automatically.

In [ ]:
# Create a coding assistant
coder = AssistantAgent(
    name="Coder",
    llm_config=llm_config,
    system_message="""You are an expert Python programmer.
    Write Python code to solve problems.
    Wrap the code in a code block that specifies the script type."""
)

# User proxy with code execution enabled
executor = UserProxyAgent(
    name="Executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # Set to True if you have Docker
    },
)

# Start coding task
executor.initiate_chat(
    coder,
    message="""Create a Python script that:
    1. Generates a list of 10 random numbers between 1 and 100
    2. Calculates their mean and standard deviation
    3. Creates a simple bar chart visualization
    Save the chart as 'random_numbers.png'"""
)

## Example 3: Group Chat with Multiple Agents

Create a group chat with specialized agents working together.

In [ ]:
# Create specialized agents
planner = AssistantAgent(
    name="Planner",
    llm_config=llm_config,
    system_message="""You are a planning expert.
    Your job is to break down complex tasks into step-by-step plans.
    Be specific and detailed in your planning."""
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""You are a software engineer.
    Follow the plan provided by the Planner and write clean, efficient code.
    Include comments and error handling."""
)

critic = AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="""You are a code reviewer.
    Review code for:
    - Correctness
    - Best practices
    - Potential bugs
    - Performance issues
    Provide constructive feedback."""
)

executor = UserProxyAgent(
    name="Executor",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "coding"},
)

# Create group chat
groupchat = GroupChat(
    agents=[executor, planner, engineer, critic],
    messages=[],
    max_round=12,
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

# Start the group chat
executor.initiate_chat(
    manager,
    message="""Create a Python function that:
    1. Takes a text file path as input
    2. Counts the frequency of each word
    3. Returns the top 10 most common words
    
    Please plan, implement, and review the solution."""
)

## Example 4: Human-in-the-Loop

Enable human feedback during agent conversations.

In [ ]:
assistant = AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
)

# User proxy that asks for human input
user_proxy_human = UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",  # Always ask for human input
    max_consecutive_auto_reply=5,
)

# Start conversation (will wait for your input)
user_proxy_human.initiate_chat(
    assistant,
    message="Help me plan a data analysis project."
)

## Example 5: Function Calling

Equip agents with custom functions they can call.

In [ ]:
# Define custom functions
def search_documents(query: str, max_results: int = 5) -> str:
    """
    Search documents in the knowledge base.
    
    Args:
        query: Search query string
        max_results: Maximum number of results to return
    
    Returns:
        Search results as a formatted string
    """
    # This would integrate with SurfSense's search functionality
    return f"""Found {max_results} documents matching '{query}':
    1. Document about AI agents and frameworks
    2. Tutorial on RAG systems
    3. Guide to vector embeddings
    4. Best practices for LLM applications
    5. Introduction to multi-agent systems"""

def get_document_content(doc_id: int) -> str:
    """
    Retrieve the content of a specific document.
    
    Args:
        doc_id: The document ID
    
    Returns:
        Document content
    """
    return f"Content of document {doc_id}: This is a sample document about AI agents..."

# Create assistant with function calling
assistant_with_functions = AssistantAgent(
    name="ResearchAssistant",
    llm_config=llm_config,
    system_message="""You are a research assistant with access to a document database.
    Use the available functions to search and retrieve information."""
)

# Register functions
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    function_map={
        "search_documents": search_documents,
        "get_document_content": get_document_content,
    },
)

# The assistant can now call these functions
user_proxy.initiate_chat(
    assistant_with_functions,
    message="Search for documents about RAG systems and summarize the key concepts."
)

## Example 6: SurfSense Integration Pattern

How to integrate AutoGen with SurfSense's backend.

In [ ]:
# Pseudo-code for SurfSense integration

class SurfSenseAgentSystem:
    """Integration of AutoGen with SurfSense."""
    
    def __init__(self, user_id: int, search_space_id: int):
        self.user_id = user_id
        self.search_space_id = search_space_id
        self.setup_agents()
    
    def setup_agents(self):
        """Initialize agents with SurfSense-specific functions."""
        
        # Create research agent with document search
        self.researcher = AssistantAgent(
            name="Researcher",
            llm_config=llm_config,
            system_message="""You are a research assistant.
            Use document search to find relevant information.
            Always cite your sources."""
        )
        
        # Create analyst agent
        self.analyst = AssistantAgent(
            name="Analyst",
            llm_config=llm_config,
            system_message="""You are a data analyst.
            Analyze information and provide insights."""
        )
        
        # Create user proxy with SurfSense functions
        self.user_proxy = UserProxyAgent(
            name="User",
            human_input_mode="NEVER",
            function_map={
                "search_documents": self.search_surfsense_documents,
                "get_document": self.get_surfsense_document,
                "create_podcast": self.create_surfsense_podcast,
            },
        )
    
    def search_surfsense_documents(self, query: str) -> str:
        """Search documents in user's SurfSense search space."""
        # Call SurfSense's hybrid search
        # return results
        pass
    
    def get_surfsense_document(self, doc_id: int) -> str:
        """Get full document content from SurfSense."""
        # Fetch document from database
        # return content
        pass
    
    def create_surfsense_podcast(self, content: str, title: str) -> str:
        """Create a podcast from the conversation."""
        # Call SurfSense podcast generation
        # return podcast_url
        pass
    
    def research_and_analyze(self, query: str) -> dict:
        """Run a research and analysis task."""
        groupchat = GroupChat(
            agents=[self.user_proxy, self.researcher, self.analyst],
            messages=[],
            max_round=10,
        )
        
        manager = GroupChatManager(
            groupchat=groupchat,
            llm_config=llm_config,
        )
        
        self.user_proxy.initiate_chat(
            manager,
            message=query,
        )
        
        return {
            "conversation": groupchat.messages,
            "summary": "Analysis complete",
        }

# Usage example
# agent_system = SurfSenseAgentSystem(user_id=123, search_space_id=1)
# result = agent_system.research_and_analyze("Analyze my research papers on AI")

## Example 7: Termination Conditions

Control when conversations should end.

In [ ]:
# Custom termination condition
def my_termination_condition(message: dict) -> bool:
    """Terminate if the word 'TERMINATE' is in the message."""
    content = message.get("content", "")
    return "TERMINATE" in content

assistant = AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    system_message="""You are a helpful assistant.
    When the task is complete, end your message with 'TERMINATE'."""
)

user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=my_termination_condition,
)

user_proxy.initiate_chat(
    assistant,
    message="Count from 1 to 5."
)

## Best Practices

1. **Clear System Messages**: Give agents specific roles and instructions
2. **Termination Conditions**: Always set clear termination conditions
3. **Error Handling**: Implement proper error handling for code execution
4. **Token Management**: Monitor token usage to control costs
5. **Security**: Use Docker for code execution in production
6. **Testing**: Test agent behaviors thoroughly before deployment
7. **Logging**: Keep track of conversations for debugging and improvement

## Advanced Topics

- **Nested Chats**: Create hierarchical agent structures
- **State Management**: Maintain context across multiple conversations
- **Custom Agents**: Extend base agent classes for specialized behavior
- **Async Operations**: Use async agents for better performance
- **RAG Integration**: Combine with retrieval-augmented generation

## Resources

- [AutoGen Official Documentation](https://microsoft.github.io/autogen/stable/)
- [AutoGen AgentChat Guide](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/)
- [AutoGen Examples](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/examples/)
- [SurfSense Documentation](https://www.surfsense.net/docs/)

## Next Steps

1. Try the examples above with your own use cases
2. Explore the group chat notebook for advanced multi-agent patterns
3. Integrate with SurfSense's document retrieval and podcast generation
4. Build your own specialized agents for your workflow
5. Experiment with different LLM models and configurations